In [1]:
import torch
import sys
from torch.utils.data import DataLoader
sys.path.append('data/')
sys.path.append('models/')
from my_dataset import MyDataset00000000 as MyDataset
from my_model import MyModel00000000 as MyModel


/Users/noelkj/miniconda3/envs/ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Parameters
data_path = '/Users/noelkj/Documents/GitHub/AI612-project2/train/testdata/'
batch_size = 32
num_epochs = 1
learning_rate = 1e-3

# Create the dataset and a data loader
dataset = MyDataset(data_path=data_path)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instantiate the model
model = MyModel()
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Create an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Loading data from /Users/noelkj/Documents/GitHub/AI612-project2/train/testdata/ to cpu


RuntimeError: Invalid magic number; corrupt file?

In [ ]:
print(model)

MyModel(
  (lstm): LSTM(128, 512, batch_first=True)
  (batchnorm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
  (task_layers): ModuleDict(
    (short_mortality): Linear(in_features=512, out_features=2, bias=True)
    (long_mortality): Linear(in_features=512, out_features=2, bias=True)
    (readmission): Linear(in_features=512, out_features=2, bias=True)
    (diagnosis): Linear(in_features=512, out_features=34, bias=True)
    (short_los): Linear(in_features=512, out_features=2, bias=True)
    (long_los): Linear(in_features=512, out_features=2, bias=True)
    (final_acuity): Linear(in_features=512, out_features=6, bias=True)
    (imminent_discharge): Linear(in_features=512, out_features=6, bias=True)
    (creatinine_level): Linear(in_features=512, out_features=5, bias=True)
    (bilirubin_level): Linear(in_features=512, out_features=5, bias=True)
    (platelet_level): Linear(in_features=512,

In [ ]:

# Training loop
for epoch in range(num_epochs):
    for batch in data_loader:
        # Unpack the data and move it to the appropriate device
        inputs = batch["data"].to(model.device)
        targets = batch["label"].to(model.device)

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = model.multitask_loss(outputs, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss for the current epoch
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {loss.item()}")

IndexError: too many indices for tensor of dimension 1